In [156]:
#  the libraries
import tensorflow as tf
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [134]:
# drawing the distribution data

In [135]:
# drawing the spectogran contrast
#  refernce : https://huggingface.co/learn/audio-course/chapter1/audio_data
# drawing the waveform
# drawing the frequency spectrum

In [136]:
def check_total_wav(base_dir): 
    genres  = os.listdir(base_dir)
    iter = 0
    for genre in range(len(genres)): 
        for file in os.listdir((os.path.join(base_dir, genres[genre]))): 
            iter += 1
        print(f'total genre {genres[genre]} is :  {iter}')
        iter = 0
check_total_wav('./data/genres_original/')

total genre blues is :  99
total genre classical is :  99
total genre country is :  99
total genre disco is :  99
total genre hiphop is :  99
total genre jazz is :  99
total genre metal is :  99
total genre pop is :  99
total genre reggae is :  99
total genre rock is :  99


In [155]:
def random_check_result_dimension():
    random = "./data/genres_original/blues/blues.00000.wav"
    audio, sr  = librosa.load(random)
                    # chroma features
    array_cft = librosa.feature.chroma_stft(y=audio, sr=sr)
    array_zero_crossing_rate = librosa.feature.zero_crossing_rate(y  = audio )
    arrray_rms = librosa.feature.rms(y = audio)
    array_spectiaL_centroid = librosa.feature.spectral_centroid(y = audio)
    array_mfcc = librosa.feature.melspectrogram(y = audio, sr = sr)
    print(np.array(array_cft).shape)
    print(np.array(array_zero_crossing_rate).shape)
    print(np.array(arrray_rms).shape)
    print(np.array(array_spectiaL_centroid).shape)
    print(np.array(array_mfcc).shape)

random_check_result_dimension()

(12, 1293)
(1, 1293)
(1, 1293)
(1, 1293)
(128, 1293)


In [223]:
def get_average(array):
    array_flat = array.flatten()
    return np.mean(array_flat)

# list most important = pass
# chroma features = pass
# zero crossing rate  = pass
# rmse = pass
# spectiaL_centroid = pass
# MFCC  = pass
# the dataset is available on : https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/data
# if neccessary
# cqt
# cens
# vqt
# melspectogram
# spectral_centroid
# spectral bandwidth 
# spectral_contrast
# the dataset is available on : https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/data

#  function for convert to excel 
def convert_excel(data_extraction: dict): 
    df = pd.DataFrame(data_extraction)
    df.to_excel("./data/extract_data/feature_extraction.xlsx")
#  succed
def load_audio_and_extract(base_dir): 
    # series_music = []
    labels = []
    iter_label = 0
    dict_features_extraction = {}
    # define array for feature extraction
    array_cft = []
    array_zero_crossing_rate = []
    arrray_rms = []
    array_spectiaL_centroid = []
    array_mfcc = []
    array_wav  = []
    # the genres
    genres = os.listdir(base_dir)
    for genre in genres: 
        genre_path = os.path.join(base_dir, genre)
        # temp_series = []  
        for music in os.listdir(genre_path):
            try: 
                array_wav.append(music)
                audio, sr  = librosa.load(os.path.join(genre_path, music))
                # chroma features
                array_cft.append(get_average(librosa.feature.chroma_stft(y=audio, sr=sr)))
                # zero crossing rate
                array_zero_crossing_rate.append(get_average(librosa.feature.zero_crossing_rate(y  = audio)))
                # rms
                arrray_rms.append(get_average(librosa.feature.rms(y = audio))) 
                # spectral centroid
                array_spectiaL_centroid.append(get_average(librosa.feature.spectral_centroid(y = audio)))
                # mel spectorgram
                array_mfcc.append(get_average(librosa.feature.melspectrogram(y = audio, sr = sr)))
                # for labels
                labels.append(iter_label)
            except Exception as e: 
                # if audio data can't load
                print(f'{os.path.join(genre_path, music)}: {e}')
        # series_music.append(temp_series)
        iter_label += 1
    
    dict_features_extraction = {
        "wav" : array_wav, 
        "chroma" : array_cft, 
        "zero crossing rate" : array_zero_crossing_rate,
        "rms" : arrray_rms, 
        "spectial centroid" : array_spectiaL_centroid,
        "mfcc" : array_mfcc, 
        "label" : labels
    }
    convert_excel(dict_features_extraction)
    # return the values
    print(f'finish convert to excel, please check to ./data/extract_data/feature_extraction.xlsx')

# load data that has been doing extraction 
def load_feature_extraction_data(data_path : str):
    # convert to excel 
    return pd.read_excel(data_path, index_col=0 ) 

# load data that has been doing extraction 
def get_train_test_data(df : pd.DataFrame): 
    features = df[df.columns[1:len(df.columns)]]
    target = df[df.columns[-1]]
    x_train, y_train, x_test, y_test = train_test_split(features, target, random_state= 42, test_size=0.1)
    return x_train, y_train, x_test, y_test 

def create_model_and_fit(x_train, y_train, x_test, y_test):
    # next : define callbacks
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units = 512, activation = 'relu',  input_shape = (6,) ), 
        tf.keras.layers.Dense(units = 256, activation = 'relu'), 
        # tf.keras.layers.Dense(units = 16, activation = 'relu'), 
        tf.keras.layers.Dense(units = 10, activation = 'softmax'), 


    ])
    model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = "adam", metrics = ['accuracy'])
    history = model.fit(x_train, y_train, epochs = 500, validation_data = (x_test,  y_test))
    return model, history 


In [ ]:
load_audio_and_extract("./data/genres_original/")
load_feature_extraction_data("./data/extract_data/feature_extraction.xlsx")

In [183]:
df = load_feature_extraction_data("./data/extract_data/feature_extraction.xlsx")
df.head()

,wav,chroma,zero crossing rate,rms,spectial centroid,mfcc,label
0,./data/genres_original/blues\blues.00000.wav,0.350129,0.083045,0.130184,1784.122641,4.425542,0
1,./data/genres_original/blues\blues.00001.wav,0.340849,0.056040,0.095908,1530.261767,2.653440,0
2,./data/genres_original/blues\blues.00002.wav,0.363538,0.076291,0.175473,1552.832481,7.253840,0
3,./data/genres_original/blues\blues.00003.wav,0.404854,0.033309,0.141040,1070.153418,6.207545,0
4,./data/genres_original/blues\blues.00004.wav,0.308526,0.101461,0.091501,1835.128513,2.106331,0


In [211]:
x_train, x_test,  y_train,y_test  = get_train_test_data(df)

In [212]:
x_train.shape

(891, 6)

In [213]:
y_train.shape

(891,)

In [224]:
create_model_and_fit(x_train, y_train, x_test, y_test )

Epoch 1/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0947 - loss: 130.5912 - val_accuracy: 0.0808 - val_loss: 41.9647
Epoch 2/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1040 - loss: 42.2569 - val_accuracy: 0.1010 - val_loss: 31.4326
Epoch 3/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0968 - loss: 33.3115 - val_accuracy: 0.1313 - val_loss: 25.7101
Epoch 4/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1019 - loss: 24.3991 - val_accuracy: 0.1010 - val_loss: 27.5911
Epoch 5/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1277 - loss: 22.8314 - val_accuracy: 0.1010 - val_loss: 12.4392
Epoch 6/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1023 - loss: 17.7715 - val_accuracy: 0.0808 - val_loss: 17.2554
Epoch 7/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1074 - loss: 16.8571 - val_accuracy: 0.1010 - val_loss: 24.8636
Epoch 8/500
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1097 - loss: 20.5453 - val_accu

(<Sequential name=sequential_11, built=True>,
 <keras.src.callbacks.history.History at 0x263558f0d10>)